In [23]:
class Node(object):
    
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        
        self.outbound_nodes = []
        
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            
        self.value = None
        
    def forward(self):
        raise NotImplementedError
        
    def backpropr(self):
        raise NotImplementedError
        
        
class Input(Node):
    
    def __init__(self):
        Node.__init__(self)
        
    def forward(self, value=None):
        if value is not None:
            self.value = value
            
class Add(Node):
    
    def __init__(self, *inputs):
        Node.__init__(self, list(inputs))
        
    def forward(self):
        self.value = 0
        
        for Input in self.inbound_nodes:
            self.value += Input.value
        
class Mult(Node):
    
    def __init__(self, *inputs):
        Node.__init__(self, list(inputs))
        
    def forward(self):
        self.value = 1
        
        for Input in self.inbound_nodes:
            self.value *= Input.value
        
def topological_sort(feed_dict):
    
    input_nodes = [n for n in feed_dict.keys()]
    
    G = {} # this is setting up the graph
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
                
    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()
        
        if isinstance(n, Input):
            n.value = feed_dict[n]
        
        L.append(n)
        
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
                
    return L
        
            
def forward_pass(output_node, sorted_nodes):
    
    for n in sorted_nodes:
        n.forward()
        
    return output_node.value

x, y = Input(), Input()

f = Add(x, y)

g = Mult(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

output = forward_pass(g, sorted_nodes)

print("{} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

10 + 5 = 15 (according to miniflow)
10 * 5 = 50 (according to miniflow)
